# Stacking Ensemble: 30 Days of ML
**Importing libraries:**

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso



**Getting Pretrained Base Learners:**
> The base learners are XGBoost and LightGBM models optimized with Optuna.

In [ ]:
df = pd.read_csv("../input/30days-folds/train_folds.csv")
df_test = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

df1 = pd.read_csv("../input/30dml-stacking-5cv/base_train_1.csv")
df2 = pd.read_csv("../input/30dml-stacking-5cv/base_train_2.csv")
df3 = pd.read_csv("../input/30dml-stacking-5cv/base_train_3.csv")

df_test1 = pd.read_csv("../input/30dml-stacking-5cv/base_test_1.csv")
df_test2 = pd.read_csv("../input/30dml-stacking-5cv/base_test_2.csv")
df_test3 = pd.read_csv("../input/30dml-stacking-5cv/base_test_3.csv")

df = df.merge(df1, on="id", how="left")
df = df.merge(df2, on="id", how="left")
df = df.merge(df3, on="id", how="left")

df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")

**Training the Meta Linear Regression model:**

In [ ]:
useful_features = ["pred_1", "pred_2", "pred_3"]
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    #model = LinearRegression(fit_intercept=False)
    model= Lasso(alpha=0.0001,precompute=True,max_iter=1000,
            positive=True, random_state=9999, selection='random', fit_intercept=True)
    

    model.fit(xtrain, ytrain)
    
    #print(model.coef_)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

In [ ]:
#0.7163095767796752

**Submitting to the leaderboard:**

In [ ]:
sample_submission.target = np.mean(np.column_stack(final_predictions), axis=1)
sample_submission.to_csv("submission.csv", index=False)